### RQ1. Does support for specific policies (carbon taxes or emissions standards) shift during elections?
Below we estimate a **Panel VAR(1)** for **RQ1**, using `cc_pol_tax` and `cc_pol_car` across 3 waves. This approach uses `panelvar`-style logic with `statsmodels` to build a simple two-variable system.

Since Python doesn’t have a native `PanelVAR` package, we'll emulate it using:
- **Lagged variables**
- **Pooled OLS** with entity dummies (or fixed effects if using `linearmodels`)

Code below will estimate the system:

$$
\begin{cases}
\text{cc\_pol\_tax}_t = \alpha_1 + \beta_{11} \cdot \text{cc\_pol\_tax}_{t-1} + \beta_{12} \cdot \text{cc\_pol\_car}_{t-1} + \epsilon_1 \\
\text{cc\_pol\_car}_t = \alpha_2 + \beta_{21} \cdot \text{cc\_pol\_tax}_{t-1} + \beta_{22} \cdot \text{cc\_pol\_car}_{t-1} + \epsilon_2
\end{cases}
$$

- **If β₁₂ is significant**: past support for car standards predicts support for carbon taxes.
- **If β₂₁ is significant**: past carbon tax support predicts future support for emissions standards.
- **Self-reinforcing effects**: significant β₁₁ or β₂₂ indicate stability over time.

In [5]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load your data
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Sort and lag the variables by person
data = data.sort_values(by=['PID', 'wave'])
data['cc_pol_tax_lag'] = data.groupby('PID')['cc_pol_tax'].shift(1)
data['cc_pol_car_lag'] = data.groupby('PID')['cc_pol_car'].shift(1)

# Drop rows where lagged data is missing (wave 2 only has lag from wave 1, which doesn't exist)
panel_var_data = data.dropna(subset=['cc_pol_tax_lag', 'cc_pol_car_lag'])

# First equation: cc_pol_tax_t ~ cc_pol_tax_{t-1} + cc_pol_car_{t-1}
X1 = panel_var_data[['cc_pol_tax_lag', 'cc_pol_car_lag']]
y1 = panel_var_data['cc_pol_tax']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Second equation: cc_pol_car_t ~ cc_pol_tax_{t-1} + cc_pol_car_{t-1}
X2 = panel_var_data[['cc_pol_tax_lag', 'cc_pol_car_lag']]
y2 = panel_var_data['cc_pol_car']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display both equations side-by-side
print(summary_col([model1, model2],
                  stars=True,
                  model_names=['cc_pol_tax_t', 'cc_pol_car_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


               cc_pol_tax_t cc_pol_car_t
----------------------------------------
const          0.6131***    1.0314***   
               (0.0770)     (0.0874)    
cc_pol_tax_lag 0.4714***    0.1933***   
               (0.0313)     (0.0279)    
cc_pol_car_lag 0.2570***    0.5362***   
               (0.0295)     (0.0315)    
R-squared      0.4282       0.4592      
R-squared Adj. 0.4276       0.4586      
N              1722         1722        
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Panel VAR Analysis – RQ1
To examine whether support for climate policies shifts during elections and whether such support exhibits dynamic interdependence over time, I estimated a Panel Vector Autoregression (Panel VAR) model with one lag (VAR(1)) using two variables: support for carbon taxes (`cc_pol_tax`) and support for emissions standards for car manufacturers (`cc_pol_car`). The analysis was conducted using data from 861 individuals observed across three waves, resulting in 2,583 panel observations.

The Panel VAR(1) model includes two equations:
1. **Support for carbon taxes at time t (`cc_pol_tax_t`)** regressed on:
   - Support for carbon taxes at time t–1 (`cc_pol_tax_lag`)
   - Support for emissions standards at time t–1 (`cc_pol_car_lag`)

2. **Support for car emissions standards at time t (`cc_pol_car_t`)** regressed on:
   - Support for car standards at time t–1 (`cc_pol_car_lag`)
   - Support for carbon taxes at time t–1 (`cc_pol_tax_lag`)

**Results**

In the first equation, both lagged predictors are statistically significant. Prior support for carbon taxes predicts current support (β = 0.4714, p < 0.01), indicating a strong self-reinforcing dynamic. In addition, prior support for emissions standards also significantly predicts current carbon tax support (β = 0.2570, p < 0.01). The model explains 42.8% of the variation in current support for carbon taxes (R² = 0.4282).

In the second equation, past support for emissions standards strongly predicts current support (β = 0.5362, p < 0.01), and past support for carbon taxes also has a positive, statistically significant effect (β = 0.1933, p < 0.01). The model explains 45.9% of the variation in emissions standards support (R² = 0.4592).

**Interpretation**

These results indicate that both policy attitudes are temporally stable and mutually reinforcing. Public support for carbon taxes and emissions standards not only persists over time but also positively influences each other. This dynamic suggests that once support for one policy begins to shift, it may contribute to broader shifts in public support for related climate policy instruments.

---

### RQ2. How does willingness to pay for climate solutions vary during elections, and what factors influence it? Are voters more likely to prioritize low-cost climate solutions during election periods?

This question has two parts:
1. **How does WTP evolve across time (especially around elections)?**
2. **What factors (e.g., harm perception, policy support) influence WTP, and vice versa?**


### What We Can Do with Panel VAR
#### What Panel VAR *can* test:
| Question                                                                                         | Panel VAR                                           |
|--------------------------------------------------------------------------------------------------|-----------------------------------------------------|
| Is WTP influenced by its **own past values**?                                                    | Yes                                                 |
| Is WTP influenced by **past values of other variables** (e.g., harm perception, policy support)? | Yes                                                 |
| Does WTP **predict** support for climate policies or harm perception?                            | Yes                                                 |
| Does the **system of attitudes shift dynamically** over time?                                    | Yes                                                 |
| Are people **more likely to prioritize low-cost solutions** during elections?                    | Not directly — would require cost-specific variable |

### How to Structure the Panel VAR for RQ2
A solid three-variable system would include:
| Variable     | Role                                             |
|--------------|--------------------------------------------------|
| `ccSolve`    | Willingness to pay                               |
| `cc4_comm`   | Perceived harm to local community                |
| `cc_pol_tax` | Support for carbon tax (policy context/attitude) |

This allows us to test:
- Does **harm perception at \( t-1 \)** increase **willingness to pay at \( t \)**?
- Does **willingness to pay at \( t-1 \)** reinforce itself?
- Does **willingness to pay at \( t-1 \)** increase future policy support?
- Does policy support affect harm perception?

## What We’ll Be Able to Conclude:

- Whether **WTP is temporally stable** (from `ccSolve_lag`)
- Whether **harm perception drives WTP**
- Whether **WTP influences policy support**, or the other way around
- Whether there's a **co-evolution** of WTP and broader climate attitudes

## What About "Low-Cost Climate Solutions"?
Panel VAR can’t test that unless we have:
- A separate variable flagging **whether the policy was low-cost**
- Or a way to decompose `ccSolve` by scenario cost level

For this reason we will make the question simpler:  
RQ2. How does willingness to pay for climate solutions vary during elections, and what factors influence it?

## Python Code for Panel VAR(1) – RQ2

In [11]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load and prep dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Sort and create lagged variables
data = data.sort_values(by=['PID', 'wave'])
data['ccSolve_lag'] = data.groupby('PID')['ccSolve'].shift(1)
data['cc4_comm_lag'] = data.groupby('PID')['cc4_comm'].shift(1)
data['cc_pol_tax_lag'] = data.groupby('PID')['cc_pol_tax'].shift(1)

# Drop missing rows
panel_var_data = data.dropna(subset=['ccSolve_lag', 'cc4_comm_lag', 'cc_pol_tax_lag'])

# Equation 1: WTP (ccSolve_t)
X1 = panel_var_data[['ccSolve_lag', 'cc4_comm_lag', 'cc_pol_tax_lag']]
y1 = panel_var_data['ccSolve']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 2: Policy support (cc_pol_tax_t)
X2 = panel_var_data[['ccSolve_lag', 'cc4_comm_lag', 'cc_pol_tax_lag']]
y2 = panel_var_data['cc_pol_tax']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 3: Harm perception (cc4_comm_t)
X3 = panel_var_data[['ccSolve_lag', 'cc4_comm_lag', 'cc_pol_tax_lag']]
y3 = panel_var_data['cc4_comm']
X3 = sm.add_constant(X3)
model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display results
print(summary_col([model1, model2, model3],
                  stars=True,
                  model_names=['ccSolve_t', 'cc_pol_tax_t', 'cc4_comm_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


               ccSolve_t cc_pol_tax_t cc4_comm_t
------------------------------------------------
const          1.0374*** 0.6629***    0.7107*** 
               (0.0675)  (0.0737)     (0.0669)  
ccSolve_lag    0.0617**  0.1164***    -0.0137   
               (0.0279)  (0.0235)     (0.0246)  
cc4_comm_lag   0.0596*** 0.1707***    0.5998*** 
               (0.0208)  (0.0204)     (0.0218)  
cc_pol_tax_lag 0.1288*** 0.5369***    0.1542*** 
               (0.0216)  (0.0256)     (0.0200)  
R-squared      0.0570    0.4324       0.4745    
R-squared Adj. 0.0554    0.4314       0.4736    
N              1722      1722         1722      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Panel VAR Analysis – RQ2
To explore how willingness to pay (WTP) for climate solutions varies over time and interacts dynamically with related attitudes, I estimated a Panel VAR(1) model including three endogenous variables:
- `ccSolve`: willingness to financially support climate action
- `cc_pol_tax`: support for a carbon tax
- `cc4_comm`: perceived harm from climate change to one's local community

Each variable was regressed on its own lag and the lagged values of the other two variables. The model was estimated using pooled OLS with standard errors clustered by respondent (`PID`) and included 1,722 observations across three waves.

### Results
**Equation 1: Willingness to Pay (`ccSolve_t`)**
| Predictor        | Coefficient | Significance |
|------------------|-------------|--------------|
| `ccSolve_lag`    | 0.062       | ** p < 0.05  |
| `cc4_comm_lag`   | 0.060       | *** p < 0.01 |
| `cc_pol_tax_lag` | 0.129       | *** p < 0.01 |
| R²               | 0.057       | —            |

→ WTP is modestly stable over time and is significantly influenced by prior **community-level harm perception** and **support for climate policy**. This suggests that both concern and policy alignment contribute to sustaining or increasing willingness to pay.

**Equation 2: Support for Carbon Tax (`cc_pol_tax_t`)**
| Predictor        | Coefficient | Significance |
|------------------|-------------|--------------|
| `ccSolve_lag`    | 0.116       | *** p < 0.01 |
| `cc4_comm_lag`   | 0.171       | *** p < 0.01 |
| `cc_pol_tax_lag` | 0.537       | *** p < 0.01 |
| R²               | 0.432       | —            |

→ Support for a carbon tax is highly stable and is dynamically influenced by both **prior WTP** and **perceived harm**. This indicates a reinforcing system in which concern and financial commitment mutually sustain support for policy.

**Equation 3: Perceived Harm to Community (`cc4_comm_t`)**
| Predictor        | Coefficient | Significance    |
|------------------|-------------|-----------------|
| `ccSolve_lag`    | –0.014      | Not significant |
| `cc4_comm_lag`   | 0.600       | *** p < 0.01    |
| `cc_pol_tax_lag` | 0.154       | *** p < 0.01    |
| R²               | 0.475       | —               |

→ Harm perception is highly self-reinforcing, and it increases with prior support for climate policy, but **not** with prior WTP. This may reflect a feedback loop in which policy attitudes shape how individuals interpret climate risks.

### Interpretation
The Panel VAR results reveal a **mutually reinforcing relationship** between willingness to pay, climate policy support, and community-level climate concern. While WTP itself exhibits only modest temporal stability, it is significantly driven by prior concern and policy support. Conversely, support for carbon taxes is highly stable and dynamically influenced by both WTP and harm perception. These findings suggest that election-period dynamics in WTP are embedded within broader patterns of **interdependent climate attitudes**, rather than occurring as sharp exogenous shifts.

Although the model does not include a direct indicator of policy cost or election period, the results suggest that **individuals' financial commitment to climate action is shaped by prior beliefs and policy support, rather than by election-period fluctuations alone**. Future models may explore low-cost vs high-cost scenarios or include election-wave interactions explicitly.

---

### RQ3. How do perceptions of (personal and community-level) climate change impacts influence policy support during elections?
> **Original research question:**  
How do perceptions of (personal and community-level) climate change impacts influence policy support during elections?

> **Panel VAR framing:**  
Are **perceived harm** and **policy support** **dynamically interrelated** across time?  
Does **past harm perception** predict **current policy support**, and vice versa?

### Suggested Variables for the Panel VAR System
To capture the core logic of RQ3:
| Variable      | Description                                                          |
|---------------|----------------------------------------------------------------------|
| `cc4_comm`    | Perceived harm to local communities                                  |
| `cc4_famheal` | Perceived harm to family health (optional: for personal-level angle) |
| `cc_pol_tax`  | Support for carbon taxes (policy support variable of focus)          |

This model lets us test:
- Whether **prior harm perceptions** (communal or personal) predict changes in **policy support**
- Whether **policy support itself** influences **future harm perceptions**, possibly via motivated reasoning
- The **bidirectional dynamics** between these constructs over time

In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Sort and create lagged variables
data = data.sort_values(by=['PID', 'wave'])
data['cc4_comm_lag'] = data.groupby('PID')['cc4_comm'].shift(1)
data['cc4_famheal_lag'] = data.groupby('PID')['cc4_famheal'].shift(1)
data['cc_pol_tax_lag'] = data.groupby('PID')['cc_pol_tax'].shift(1)

# Drop rows with missing values
panel_var_data = data.dropna(subset=['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag'])

# Equation 1: policy support as outcome
X1 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag']]
y1 = panel_var_data['cc_pol_tax']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 2: community-level harm as outcome
X2 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag']]
y2 = panel_var_data['cc4_comm']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 3: family-level harm as outcome
X3 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag']]
y3 = panel_var_data['cc4_famheal']
X3 = sm.add_constant(X3)
model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display results
print(summary_col([model1, model2, model3],
                  stars=True,
                  model_names=['cc_pol_tax_t', 'cc4_comm_t', 'cc4_famheal_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


                cc_pol_tax_t cc4_comm_t cc4_famheal_t
-----------------------------------------------------
const           0.7809***    0.6603***  0.4720***    
                (0.0719)     (0.0611)   (0.0588)     
cc4_comm_lag    0.1402***    0.4848***  0.3034***    
                (0.0327)     (0.0323)   (0.0339)     
cc4_famheal_lag 0.0478       0.1427***  0.3372***    
                (0.0296)     (0.0293)   (0.0350)     
cc_pol_tax_lag  0.5511***    0.1484***  0.1244***    
                (0.0251)     (0.0199)   (0.0202)     
R-squared       0.4260       0.4816     0.4543       
R-squared Adj.  0.4250       0.4807     0.4533       
N               1722         1722       1722         
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Panel VAR Analysis – RQ3 (Carbon Tax as Outcome)
To explore the dynamic relationship between perceived climate change harm and support for climate policy, I estimated a Panel VAR(1) model including three variables:
- `cc_pol_tax`: support for a carbon tax  
- `cc4_comm`: perceived harm from climate change to one's local community  
- `cc4_famheal`: perceived harm to one’s family’s health

Each variable was regressed on its own lag and the lagged values of the other two variables. The model was estimated using pooled OLS with clustered standard errors by individual (PID), covering three waves and 1,722 complete observations.

### Results
**Equation 1: Support for Carbon Tax (`cc_pol_tax_t`)**  
| Predictor | Coefficient | Significance |
|-----------|-------------|--------------|
| `cc4_comm_lag` | **0.140** | *** p < 0.01 |
| `cc4_famheal_lag` | 0.048 | Not significant |
| `cc_pol_tax_lag` | **0.551** | *** p < 0.01 |

→ Support for carbon taxes is strongly self-reinforcing over time. Importantly, perceptions of **community-level harm** significantly increase future support for carbon taxes, while **family health concerns** do not show a significant direct effect.

**Equation 2: Perceived Harm to Community (`cc4_comm_t`)**  
| Predictor | Coefficient | Significance |
|-----------|-------------|--------------|
| `cc4_comm_lag` | **0.485** | *** p < 0.01 |
| `cc4_famheal_lag` | **0.143** | *** p < 0.01 |
| `cc_pol_tax_lag` | **0.148** | *** p < 0.01 |

→ Perceptions of community-level harm are highly stable and also shaped by **prior support for climate policy**, suggesting a feedback mechanism where prior endorsement of action may increase perceptions of harm.

**Equation 3: Perceived Harm to Family Health (`cc4_famheal_t`)**  
| Predictor | Coefficient | Significance |
|-----------|-------------|--------------|
| `cc4_comm_lag` | **0.303** | *** p < 0.01 |
| `cc4_famheal_lag` | **0.337** | *** p < 0.01 |
| `cc_pol_tax_lag` | **0.124** | *** p < 0.01 |

→ Perceived harm to family health is also self-reinforcing and shaped by both other variables. Prior perceptions of harm to the community have a strong influence on subsequent perceptions of personal harm, as does support for climate policy.

### Interpretation
This Panel VAR model shows that both **community-level and personal-level climate harm perceptions** are dynamically linked with **policy support**. Community-level harm perception appears particularly influential in predicting **future support for carbon taxes**. At the same time, **policy support also contributes to increased harm perception**, suggesting a reciprocal dynamic.

These findings are consistent with theoretical models of climate opinion formation, where perceptions and policy preferences evolve together. Notably, while both forms of perceived harm are internally stable, **community-level concern appears more strongly connected to policy attitudes** than personal-level concern.

---

We will now estimate the exact same **Panel VAR(1)** model with `cc_pol_car` (support for emissions standards) as the policy support variable, replacing `cc_pol_tax`. This allows us to directly compare whether perceived harm affects **support for regulatory solutions** (like car standards) differently from **market-based solutions** (like carbon taxes).

### Variables for the Updated System
| Variable      | Description                                   |
|---------------|-----------------------------------------------|
| `cc_pol_car`  | Support for emissions standards (new outcome) |
| `cc4_comm`    | Perceived harm to local community             |
| `cc4_famheal` | Perceived harm to family health               |

We will create lagged versions of all three and estimate the same three equations system.

In [6]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Sort and create lagged variables
data = data.sort_values(by=['PID', 'wave'])
data['cc4_comm_lag'] = data.groupby('PID')['cc4_comm'].shift(1)
data['cc4_famheal_lag'] = data.groupby('PID')['cc4_famheal'].shift(1)
data['cc_pol_car_lag'] = data.groupby('PID')['cc_pol_car'].shift(1)

# Drop missing rows
panel_var_data = data.dropna(subset=['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag'])

# Equation 1: cc_pol_car_t
X1 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag']]
y1 = panel_var_data['cc_pol_car']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 2: cc4_comm_t
X2 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag']]
y2 = panel_var_data['cc4_comm']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 3: cc4_famheal_t
X3 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag']]
y3 = panel_var_data['cc4_famheal']
X3 = sm.add_constant(X3)
model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display results
print(summary_col([model1, model2, model3],
                  stars=True,
                  model_names=['cc_pol_car_t', 'cc4_comm_t', 'cc4_famheal_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


                cc_pol_car_t cc4_comm_t cc4_famheal_t
-----------------------------------------------------
const           0.9345***    0.5058***  0.4855***    
                (0.0857)     (0.0696)   (0.0698)     
cc4_comm_lag    0.1431***    0.4792***  0.3195***    
                (0.0294)     (0.0327)   (0.0341)     
cc4_famheal_lag 0.0196       0.1466***  0.3421***    
                (0.0263)     (0.0292)   (0.0349)     
cc_pol_car_lag  0.6020***    0.1701***  0.0870***    
                (0.0244)     (0.0214)   (0.0217)     
R-squared       0.4613       0.4840     0.4475       
R-squared Adj.  0.4604       0.4831     0.4466       
N               1722         1722       1722         
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Panel VAR Analysis – RQ3 (Emissions Standards as Outcome)
To complement the prior analysis of support for carbon taxes, I estimated a Panel VAR(1) model using `cc_pol_car` (support for stronger emissions standards for car manufacturers) as the primary policy support outcome. The model includes two perceived harm variables, `cc4_comm` (harm to local communities) and `cc4_famheal` (harm to family health), as endogenous variables in the system. Each variable was regressed on its own lag and the lags of the other two. The model was estimated using pooled OLS with clustered standard errors by respondent (PID) and includes 1,722 observations from three waves.

### Results
**Equation 1: Support for Emissions Standards (`cc_pol_car_t`)**  
| Predictor          | Coefficient | Significance    |
|--------------------|-------------|-----------------|
| `cc4_comm_lag`     | **0.143**   | *** p < 0.01    |
| `cc4_famheal_lag`  | 0.020       | Not significant |
| `cc_pol_car_lag`   | **0.602**   | *** p < 0.01    |
| R²                 | 0.461       | —               |

→ Support for emissions standards is highly stable over time and significantly influenced by previous perceptions of harm to the local community. Perceived harm to family health does not show a significant direct effect on this form of policy support.

**Equation 2: Perceived Harm to Local Community (`cc4_comm_t`)**  
| Predictor          | Coefficient | Significance |
|--------------------|-------------|--------------|
| `cc4_comm_lag`     | **0.479**   | *** p < 0.01 |
| `cc4_famheal_lag`  | **0.147**   | *** p < 0.01 |
| `cc_pol_car_lag`   | **0.170**   | *** p < 0.01 |
| R²                 | 0.484       | —            |

→ Community-level harm perception is highly stable and positively influenced by previous policy support and perceptions of personal harm.

**Equation 3: Perceived Harm to Family Health (`cc4_famheal_t`)**  
| Predictor          | Coefficient | Significance |
|--------------------|-------------|--------------|
| `cc4_comm_lag`     | **0.320**   | *** p < 0.01 |
| `cc4_famheal_lag`  | **0.342**   | *** p < 0.01 |
| `cc_pol_car_lag`   | **0.087**   | *** p < 0.01 |
| R²                 | 0.448       | —            |

→ Perceptions of harm to family health are influenced by both their own lagged values and by prior community-level harm perceptions. Support for emissions standards also has a modest positive effect on subsequent personal harm perceptions.

### Interpretation
Support for emissions standards is not only self-reinforcing but also significantly predicted by prior perceptions of harm to local communities. These findings mirror the patterns observed in the carbon tax model, suggesting that **community-level concern** is a more robust and consistent predictor of policy support than **personal-level concern**. Moreover, the feedback effects from policy support to harm perceptions highlight the potential role of motivated reasoning or belief updating over time.

---

### RQ4. Does political affiliation or ideology moderate the relationship between perceptions of harm and willingness to pay?
> **RQ4:** Does political affiliation or ideology **moderate** the relationship between perceptions of harm and willingness to pay?

This is a **moderation question**, asking:
- Is the effect of **harm perception** on **WTP** different depending on **political identity**?

#### Moderation ≠ What Panel VAR Does
Panel VAR estimates **dynamic feedback** between variables over time, **treating everything as endogenous**. It doesn’t easily accommodate:
- **Time-invariant moderators** (like `pol_ideology`)
- **Interaction terms** (e.g., `cc4_comm × ideology`)

### What We *Can* Do with Panel VAR for RQ4
We *can* estimate a Panel VAR system with:
- `ccSolve` (willingness to pay)
- `cc4_comm` (harm perception)

and **run it separately by ideological group** (e.g., liberals vs conservatives).

This gives us a **subgrouped Panel VAR**, which lets us explore:

> "Is the dynamic link between harm perception and WTP different across ideological groups?"

### How to Do It: Subgrouped Panel VAR
#### 1. Create ideology groups and run separate Panel VARs for each group
#### 2. Compare the coefficients for `cc4_comm_lag` in each group
- If it's **larger among conservatives**, harm perception influences WTP more strongly there.
- If it's **larger among liberals**, it matters more for them.

In [8]:
data['ideology_group'] = data['pol_ideology'].apply(
    lambda x: 'liberal' if x >= 4 else ('conservative' if x <= 2 else 'moderate')
)

for group in ['conservative', 'liberal']:
    subgroup = data[data['ideology_group'] == group].copy()
    subgroup = subgroup.sort_values(by=['PID', 'wave'])

    # Create lags
    subgroup['ccSolve_lag'] = subgroup.groupby('PID')['ccSolve'].shift(1)
    subgroup['cc4_comm_lag'] = subgroup.groupby('PID')['cc4_comm'].shift(1)
    subgroup = subgroup.dropna(subset=['ccSolve_lag', 'cc4_comm_lag'])

    # Model 1: WTP as outcome
    X1 = subgroup[['ccSolve_lag', 'cc4_comm_lag']]
    y1 = subgroup['ccSolve']
    X1 = sm.add_constant(X1)
    model = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': subgroup['PID']})

    print(f"\nSubgroup: {group.upper()}")
    print(model.summary())


Subgroup: CONSERVATIVE
                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     12.20
Date:                Sun, 13 Apr 2025   Prob (F-statistic):           8.39e-06
Time:                        18:33:25   Log-Likelihood:                -606.18
No. Observations:                 516   AIC:                             1218.
Df Residuals:                     513   BIC:                             1231.
Df Model:                           2                                         
Covariance Type:              cluster                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.1229 

### Panel VAR Analysis – RQ4 (Subgrouped by Political Ideology)
To assess whether political ideology moderates the relationship between climate harm perception and willingness to pay (WTP) for climate solutions, I estimated separate Panel VAR(1) models for conservative and liberal participants. The key dependent variable was `ccSolve` (willingness to pay), with lagged predictors including `ccSolve_lag` and `cc4_comm_lag` (perceived harm to local communities).

Models were estimated using pooled OLS with standard errors clustered at the individual level (`PID`).

### Results
#### Conservative Participants (pol_ideology = 1–2)
| Predictor       | Coefficient | Significance    |
|-----------------|-------------|-----------------|
| `cc4_comm_lag`  | **0.129**   | *** p < 0.01    |
| `ccSolve_lag`   | 0.085       | Not significant |
| R-squared       | 0.052       | —               |

→ Among conservatives, **perceived harm to the local community at t–1 significantly increases willingness to pay at time t**. This suggests that harm perception is a strong predictor of climate-related financial support in this subgroup.

#### Liberal Participants (pol_ideology = 4–5)

| Predictor       | Coefficient | Significance |
|-----------------|-------------|--------------|
| `cc4_comm_lag`  | 0.003       | Not significant |
| `ccSolve_lag`   | **0.154**   | ** p < 0.01 |
| R-squared       | 0.022       | —            |

→ Among liberals, **willingness to pay is primarily predicted by its own past value**, with **no significant contribution from perceived harm**. This suggests that liberals’ willingness to pay is more stable and less influenced by short-term shifts in perceived community-level harm.

### Interpretation
These findings provide evidence that **political ideology moderates the relationship between harm perception and willingness to pay**. Among conservatives, harm perception plays a **significant causal role** in increasing willingness to pay for climate solutions, whereas among liberals, WTP appears **less sensitive to harm perception** and more **internally consistent over time**.

This is consistent with theoretical expectations: liberals may hold a more **ideologically grounded** climate position, whereas conservative support may depend more on **issue salience** or **personal/community relevance**.